In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sps
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook as tqdm
import pickle
import os.path

from collections import defaultdict

# I. Класс реализованного метода

In [4]:
class UserBasedRecommender():
    def __init__(self, path_to_orders, path_to_order_products_train, decomposition_dim):
        '''
        :param path_to_orders: путь к исходному датафрейму с keggle orders.csv
        :param path_to_order_products_train: путь к новому тренировочному датафрейму, полученному в п.II ниже
        :param decomposition_dim: размерность декомпозиции
        '''
        self.path_to_orders = path_to_orders
        self.path_to_order_products_train = path_to_order_products_train
        self.n_comp = decomposition_dim
        
        self.ok_status = True
        check, message = self.check_input_params()
        
        if not check:
            self.ok_status = False
            print(message)
    
    def check_input_params(self):
        if not os.path.isfile(self.path_to_orders):
            return False, 'File doesnt exist'
        if not os.path.isfile(self.path_to_order_products_train):
            return False, 'File doesnt exist'
        if not isinstance(self.n_comp, int) or not self.n_comp > 0:
            return False, 'decomposition_dim should be positive integer value'
        
        return True, 'Ok'
            
    def read_data(self):
        if not self.ok_status:
            return 'There were some problems'
        
        self.orders = pd.read_csv(self.path_to_orders)
        self.order_products_train = pd.read_csv(self.path_to_order_products_train)
    
    def prepare_data(self):
        '''
        создает таблицу user-item для дальнейших манипуляций
        '''
        if not self.ok_status:
            return 'There were some problems'
        
        self.merged = pd.merge(self.orders, self.order_products_train, on='order_id', how='right')
        self.merged = self.merged[['order_id', 'user_id', 'product_id']]
        
        self.user_id_num = {key: value for value, key in enumerate(self.merged['user_id'].unique(), start=0)}
        self.product_id_num = {key: value for value, key in enumerate(self.merged['product_id'].unique(), start=0)}
        self.product_num_id = {value: key for value, key in enumerate(self.merged['product_id'].unique(), start=0)}
        
        self.merged['user_num'] = self.merged['user_id'].transform(lambda x: self.user_id_num[x])
        self.merged['product_num'] = self.merged['product_id'].transform(lambda x: self.product_id_num[x])
        self.merged['buy'] = 1
        
        # матрица купил/не купил
        self.user_item_matrix = sps.coo_matrix((self.merged.buy, (self.merged.user_num, self.merged.product_num)))
        
        full = self.merged.groupby(by=['user_num', 'product_num']).count()['product_id']
        
        self.user_item_count_matrix = sps.coo_matrix((full.values, (full.index.get_level_values(0).values, \
                                                                    full.index.get_level_values(1).values)))
        
    def decompose(self):
        '''
        декомпозиция user-item_matrix с размерностью n_comp
        '''
        if not self.ok_status:
            return 'There were some problems'
        
        self.model = NMF(n_components=self.n_comp, init='random', random_state=0)
        self.user_matrix = self.model.fit_transform(self.user_item_matrix)
        self.product_matrix = self.model.components_
    
    def get_similarity(self, top_n_size):
        '''Каждому юзеру сопоставляет id n-схожих юзеров
        :param top_n_size: сколько схожих пользователей по убыванию схожести надо сохранить
        '''
        if not self.ok_status:
            return 'There were some problems'
        
        
        self.similar_users_df = pd.DataFrame()
        self.similar_users_weight_df = pd.DataFrame()
            
        size = 100
        users_cnt = len(self.user_matrix)
        add = 1 if users_cnt % size > 0 else 0
        parts_cnt = users_cnt//size + add

        for i in tqdm(range(parts_cnt)):
            start = size * i
            end = size * i + size
            
            if end > 2000:
                break
                
            user_to_others_similarity = cosine_similarity(Y=self.user_matrix, X=self.user_matrix[start:end])
                
            sim_users_df = pd.DataFrame(np.argsort(-user_to_others_similarity)).loc[:, :top_n_size]
            sim_users_weight_df = pd.DataFrame(-np.sort(-user_to_others_similarity)).loc[:, :top_n_size]
                
            self.similar_users_df = pd.concat([self.similar_users_df, sim_users_df], ignore_index=True)
            self.similar_users_weight_df = pd.concat([self.similar_users_weight_df, sim_users_weight_df], ignore_index=True)
    
    def save_similarity_df(self, path_users, path_weights):
        if not self.ok_status:
            return 'There were some problems'
        
        self.similar_users_df.to_csv(path_users, index=False)
        self.similar_users_weight_df.to_csv(path_weights, index=False)
        
    def set_similarity_df(self, path_to_df_users, path_to_df_weights):
        '''
        Если уже есть посчитанная матрица, то можем использовать и обойти долгий кусок обучения в fit(),
        указав параметр loaded_df=True
        :param path_to_df: путь до csv файла с учебным датафреймом
        '''
        if os.path.isfile(path_to_df_users) and os.path.isfile(path_to_df_weights):
            self.similar_users_df = pd.read_csv(path_to_df_users)
            self.similar_users_weight_df = pd.read_csv(path_to_df_weights)
        else:
            # оформить ошибки по-человечески - нужен же возврат к True
            # self.ok_status = False
            print('File doesnt exist')
    
    def fit(self, path_to_save_users=False, path_to_save_weights=False, loaded_df=False, top_n_size=5):
        '''
        :param self:
        :param path_to_save: путь для сохранения матрицы сх. юзеров и весов (названия дефолтные)
        :param loaded_df: bool - True если матрица уже была загружена и пересчитывать заново ее не нужно;
                          default=False
        :param top_n_size: размер топ схожих пользователей 
        :return: информация в случае ошибки 
        '''
        if not self.ok_status:
            return 'There were some problems'
        
        self.top_n_size = top_n_size
        
        self.read_data()     # читает необходимые df
        self.prepare_data()  # создание таблицы user-item с необх. полями и разметками
        if not loaded_df:
            self.decompose()     # декомпозиция матрицы на 2 матрицы размерности n
            self.get_similarity(self.top_n_size)
            if path_to_save_users and path_to_save_weights:
                self.save_similarity_df(path_to_save_users, path_to_save_weights)
    
    def predict(self, user_id=False, sim_users_count=False, users_count=False):
        '''
        :param self:
        :param user_id:  id юзера, для которого считаем предсказание покупок, если False, то считаем всех
        :param sim_users_count: если нужно учесть не весь топ схожих юзеров, а только его часть, 
                                равную значению данного параметра
        :param users_count: если нужно посчитать не для одного/всех юзеров, а для части, 
                                равной значению данного параметра
        :return: сообщение в случае ошибки; в случае успеха - словарь {user_id: products_id}
        '''
        if not self.ok_status:
            return 'There were some problems'
        
        if not sim_users_count or sim_users_count > self.top_n_size:
            sim_users_count = self.top_n_size
        
        if user_id:
            if user_id in self.user_id_num:
                users_to_check = [user_id] 
            else:
                return 'There is no user with this id'
        else:
            users_to_check = [*self.user_id_num][:users_count] if users_count else [*self.user_id_num]
            
        self.predicted_products = dict()
            
        users_purchases = pd.Series.sparse.from_coo(self.user_item_matrix)
        

        
        # проход по всем юзерам, для которых хотим получить предсказание покупок
        for user_id in tqdm(users_to_check):
            user_num = self.get_user_num(user_id)
            
            # номера и веса схожих c user_id пользователей
            sim_users_nums = self.similar_users_df.loc[user_num].values
            sim_users_weights = self.similar_users_weight_df.loc[user_num].values

            d = defaultdict(int)
            
            # проход по всем юзерам схожим с предсказываемым
            for sim_user_num, sim_user_weight in zip(sim_users_nums[:sim_users_count], sim_users_weights[:sim_users_count]):
            
                sim_user_purchases = users_purchases[sim_user_num].index
                
                # к каждому купленному схожим юзером продукту прибавляем его вес относительно предсказываемого
                for product in sim_user_purchases:
                        d[product] += sim_user_weight

            d_ids = defaultdict(int)

            # ключами были номера продуктов - заменим на id
            for key in d:
                d_ids[self.product_num_id[key]] = d[key]
                
            self.predicted_products[user_id] = [*dict(sorted(d_ids.items(), key=lambda x: x[1], reverse=True))]
        return self.predicted_products
    
    def predict_by_count(self, user_id=False, sim_users_count=False, users_count=False, products_top_size=10):
        '''
        отличается от predict тем, что учитывает кол-во купленного и рассматривает у каждого схожего юзера только top-n продуктов
        
        :param self:
        :param user_id:  id юзера, для которого считаем предсказание покупок, если False, то считаем всех
        :param sim_users_count: если нужно учесть не весь топ схожих юзеров, а только его часть, 
                                равную значению данного параметра
        :param users_count: если нужно посчитать не для одного/всех юзеров, а для части, 
                                равной значению данного параметра
        :return: сообщение в случае ошибки; в случае успеха - словарь {user_id: products_id}
        '''
        if not self.ok_status:
            return 'There were some problems'
        
        if not sim_users_count or sim_users_count > self.top_n_size:
            sim_users_count = self.top_n_size
        
        if user_id:
            if user_id in self.user_id_num:
                users_to_check = [user_id] 
            else:
                return 'There is no user with this id'
        else:
            users_to_check = [*self.user_id_num][:users_count] if users_count else [*self.user_id_num]
            
        self.predicted_products = dict()
            
        users_purchases = pd.Series.sparse.from_coo(self.user_item_count_matrix)
        
        # проход по всем юзерам, для которых хотим получить предсказание покупок
        for user_id in tqdm(users_to_check):
            user_num = self.get_user_num(user_id)
            
            # номера и веса схожих c user_id пользователей
            sim_users_nums = self.similar_users_df.loc[user_num].values
            sim_users_weights = self.similar_users_weight_df.loc[user_num].values

            d = defaultdict(int)
            
            # проход по всем юзерам схожим с предсказываемым
            for sim_user_num, sim_user_weight in zip(sim_users_nums[:sim_users_count], sim_users_weights[:sim_users_count]):
                
                if sim_user_num == user_num:
                    continue
                
                sim_user_purchases = dict(users_purchases[sim_user_num])
                
                # к каждому продукта из топ-n купленных схожим юзером прибавляем его вес относительно предсказываемого
                for product, count in sorted(sim_user_purchases.items(), key=lambda x: x[1], reverse=True)[:products_top_size]:
                        d[product] += sim_user_weight

            d_ids = defaultdict(int)

            # ключами были номера продуктов - заменим на id
            for key in d:
                d_ids[self.product_num_id[key]] = d[key]
                
            self.predicted_products[user_id] = [*dict(sorted(d_ids.items(), key=lambda x: x[1], reverse=True))]
        return self.predicted_products
    
    def get_coo(self):
        return self.user_item_matrix
    
    def get_user_num(self, user_id):
        return self.user_id_num[user_id]
    
    def prepare_validation_data(self, path_to_valid_df):
        '''
        создает таблицу user-item для дальнейших манипуляций
        :param path_to_valid_df: путь до csv таблицы с валидационным датафреймом
        '''
        if not self.ok_status:
            return 'There were some problems'
        
        self.order_products_valid = pd.read_csv(path_to_valid_df)
        
        self.merged_valid = pd.merge(self.orders, self.order_products_valid, on='order_id', how='right')
        self.merged_valid = self.merged_valid[['order_id', 'user_id', 'product_id']]
        
        self.user_id_num_valid = {key: value for value, key in enumerate(self.merged_valid['user_id'].unique(), start=0)}
        self.product_id_num_valid = {key: value for value, key in enumerate(self.merged_valid['product_id'].unique(), start=0)}
        self.product_num_id_valid = {value: key for value, key in enumerate(self.merged_valid['product_id'].unique(), start=0)}
        
        self.merged_valid['user_num'] = self.merged_valid['user_id'].transform(lambda x: self.user_id_num_valid[x])
        self.merged_valid['product_num'] = self.merged_valid['product_id'].transform(lambda x: self.product_id_num_valid[x])
        self.merged_valid['buy'] = 1
        
        self.user_item_matrix_valid = sps.coo_matrix((self.merged_valid.buy, (self.merged_valid.user_num, self.merged_valid.product_num)))
        
    def get_data_to_check_metrics(self, users_count=False, sim_users_count=False, products_top_size=10):
        '''
        :param count_users: для скольких юзеров посчитаем метрики
        :param sim_users_count: сколько юзеров из топа схожих будут учитываться для предсказания покупок
        '''
        predicted_purchases = {}
        actual_purchases = {}
        
        # predicted = self.predict(sim_users_count=sim_users_count, users_count=users_count)
        predicted = self.predict_by_count(sim_users_count=sim_users_count, users_count=users_count, products_top_size=products_top_size)
        
        users_purchases_valid = pd.Series.sparse.from_coo(self.user_item_matrix_valid)
        
        for user_id in predicted:
            if user_id in self.user_id_num_valid:
                user_num_in_valid_df = self.user_id_num_valid[user_id]
                user_actual_purchases_nums = users_purchases_valid[user_num_in_valid_df].index
                    
                actual = [self.product_num_id_valid[product_num] for product_num in user_actual_purchases_nums]
            
                predicted_purchases[user_id] = predicted[user_id]
                actual_purchases[user_id] = actual
            
            else:
                print('No user with id: ', user_id)
        
        return actual_purchases, predicted_purchases

# II. Пропишем все пути и сконструируем датасеты мечты

In [1]:
import pickle

# необходимые файлы с исходными датасетами с keggle
path_data_train = 'D:/Projects/Anaconda/EDA/data/order_products__train.csv'
path_data_prior = 'D:/Projects/Anaconda/EDA/data/order_products__prior.csv'
path_data_orders = 'D:/Projects/Anaconda/EDA/data/orders.csv'
    
# [куда запишем]/[где уже лежат] файлы с индексами
train_path = 'data_split/train_indices.pickle'
valid_path = 'data_split/validation_indices.pickle'
test_path = 'data_split/test_indices.pickle'

# куда запишем файлы с новым разделением
# df_train_path = 'data_split/train_data.csv'
# df_valid_path = 'data_split/validation_data.csv'
# df_test_path = 'data_split/test_data.csv'

# куда запишем файлы с новым разделением
df_train_path = 'data_split/train_1_data.csv'
df_valid_path = 'data_split/validation_1_data.csv'
df_test_path = 'data_split/test_1_data.csv'

### Если файлов с индексами нет

In [267]:
def _split_indices(grouped_ratings, retriever):
    return np.concatenate(grouped_ratings.apply(retriever).values)

def split(path_to_orders):
    orders = pd.read_csv(path_to_orders)
    grouper = orders.sort_values('order_number').groupby('user_id')
    train_indices = _split_indices(
        grouper,
        lambda user_ratings: user_ratings[:int(user_ratings.shape[0] * 0.5)].index.values)
    
    validation_indices = _split_indices(
        grouper,
        lambda user_ratings: user_ratings.iloc[int(user_ratings.shape[0] * 0.5):
                                               int(user_ratings.shape[0] * 0.75)].index.values)
    
    test_indices = _split_indices(
        grouper,
        lambda user_ratings: user_ratings.iloc[int(user_ratings.shape[0] * 0.75):].index.values)
    
    return train_indices, validation_indices, test_indices


train_indices, validation_indices, test_indices = split(path_data_orders)

# save results
with open(train_path, 'wb') as out:
    pickle.dump(train_indices, out)

with open(valid_path, 'wb') as out:
    pickle.dump(validation_indices, out)

with open(test_path, 'wb') as out:
    pickle.dump(test_indices, out)

### Файлы с индексами готовы - теперь скомпонуем и запишем в файл все необходимое

In [4]:
def read_and_concat_same_data(df_train_path, df_prior_path, df_orders_path):
    
    df_train = pd.read_csv(df_train_path)
    df_prior = pd.read_csv(df_prior_path)
    df_orders = pd.read_csv(df_orders_path)
    
    order_products_full = pd.concat([df_train, df_prior])
    
    return order_products_full, df_orders


def get_split_df(train_path, valid_path, test_path, df_orders, df_order_products_full):
    
    with open(train_path, 'rb') as input:
        train_indices = pickle.load(input)

    with open(valid_path, 'rb') as input:
        validation_indices = pickle.load(input)

    with open(test_path, 'rb') as input:
        test_indices = pickle.load(input)
        
    train_order_ids = df_orders.loc[train_indices]['order_id'].values
    validation_order_ids = df_orders.loc[validation_indices]['order_id'].values
    test_order_ids = df_orders.loc[test_indices]['order_id'].values
    
    train_df = df_order_products_full[df_order_products_full['order_id'].isin(train_order_ids)]
    validation_df = df_order_products_full[df_order_products_full['order_id'].isin(validation_order_ids)]
    test_df = df_order_products_full[df_order_products_full['order_id'].isin(test_order_ids)]
    
    return train_df, validation_df, test_df


# считается что уже имеем 3 файла с индексами
df_order_products_full, df_orders = read_and_concat_same_data(path_data_train, path_data_prior, path_data_orders)
train_df, validation_df, test_df = get_split_df(train_path, valid_path, test_path, df_orders, df_order_products_full)

# write new df's to file:
train_df.to_csv(df_train_path, index=False)
validation_df.to_csv(df_valid_path, index=False)
test_df.to_csv(df_test_path, index=False)

# IV. Посчитаем с новым алгоритмом

### Пример с обучением

In [5]:
# файл, куда по необходимости загрузим результаты
path_to_res_users = 'data/new_df_res_top_100_dim_200_users.csv'
path_to_res_weights = 'data/new_df_res_top_100_dim_200_weights.csv'
# необходимые файлы: 
# 1. исходный из keggle orders
path_to_orders = 'data/orders.csv'

# 2. созданный-смерженный как надо методами из п.II
df_train_path = 'data_split/train_1_data.csv'

# размерность для декомпозиции
n_comp = 200

# top_n
top_n_size = 100

# создаем экземпляр класса с вышеописанными параметрами
Recommender = UserBasedRecommender(path_to_orders, df_train_path, n_comp)

Recommender.fit(path_to_save_users=path_to_res_users, path_to_save_weights=path_to_res_weights, top_n_size=top_n_size)

# ПОСЧИТАНО ТОЛЬКО ДЛЯ 2-Х ТЫСЯЧ

D:\Soft\Anaconda\lib\site-packages\ipykernel_launcher.py:89: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


### Пример без обучения (просто загрузка рез-тов)

In [86]:
# файл, куда по необходимости загрузим результаты
path_to_res_users = 'data/new_df_res_top_500_dim_35_users.csv'
path_to_res_weights = 'data/new_df_res_top_500_dim_35_weights.csv'
# необходимые файлы: 
# 1. исходный из keggle orders
path_to_orders = 'data/orders.csv'

# 2. созданный-смерженный как надо методами из п.II
df_train_path = 'data_split/train_1_data.csv'

# размерность для декомпозиции
n_comp = 35

# top_n
top_n_size = 500

# создаем экземпляр класса с вышеописанными параметрами
Recommender = UserBasedRecommender(path_to_orders, df_train_path, n_comp)

# Загружаем уже посчитанные результаты
Recommender.set_similarity_df(path_to_res_users, path_to_res_weights)

Recommender.fit(loaded_df=True, top_n_size=top_n_size)

In [24]:
# посчитаем для 15 юзеров и учтем только топ-3 схожих (3 наиболее похожих юзера для каждого)
Recommender.predict(user_id=1, sim_users_count=3)

D:\Soft\Anaconda\lib\site-packages\ipykernel_launcher.py:178: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


{1: [196,
  46149,
  12427,
  13176,
  10258,
  25133,
  38300,
  26088,
  26405,
  14084,
  16732,
  31102,
  25286,
  13032,
  30450,
  10326,
  17122,
  41787,
  22362,
  15680,
  46521,
  1,
  41276]}

In [23]:
# посчитаем для 15 юзеров и учтем только топ-3 схожих (3 наиболее похожих юзера для каждого)
Recommender.predict_by_count(user_id=125, sim_users_count=3)

D:\Soft\Anaconda\lib\site-packages\ipykernel_launcher.py:236: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


{125: [29487,
  31564,
  39481,
  33548,
  48205,
  23296,
  15860,
  35372,
  30162,
  38200,
  20244,
  24749,
  1217,
  40925,
  831,
  44156,
  7883,
  10312,
  42441,
  37766]}

# Метрички

In [ ]:
# Рассматривать большое число соседей не имеет смысла, так как результат либо тот же либо сильно хуже
# 5-15 довольно оптимальные числа и по времени занимает меньше, чем предсказывать покупки по 100 схожим юзерам

# Размерность декомпозиции тоже влияет, с dim=50 результаты в целом получше, чем с 10, 
# но считалось это в разы дольше - может имеет смысл взять около 25

In [7]:
# готовим данные для проверки метрик
Recommender.prepare_validation_data('data_split/validation_1_data.csv')

In [18]:
# получаем данные предсказанное/реально купленное
predicted, actual = Recommender.get_data_to_check_metrics(users_count=100, sim_users_count=50, products_top_size=5)

D:\Soft\Anaconda\lib\site-packages\ipykernel_launcher.py:236: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [13]:
# AP@k
def apk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

# MAP@K
def mapk(actual, predicted, k=10):
    # return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])
    return [apk(a,p,k) for a,p in zip(actual, predicted)]

In [20]:
mapk(predicted.values(), actual.values(), k=10)

[0.19666666666666666,
 0.15,
 0.24952380952380948,
 0.0,
 0.1,
 0.0,
 0.17666666666666667,
 0.2375,
 0.13333333333333333,
 0.05,
 0.05,
 0.21666666666666665,
 0.2,
 0.065,
 0.20396825396825397,
 0.2544444444444444,
 0.1,
 0.0,
 0.05,
 0.7761904761904761,
 0.2095238095238095,
 0.06666666666666667,
 0.0,
 0.1,
 0.0,
 0.0,
 0.2,
 0.12333333333333334,
 0.0,
 0.5,
 0.05,
 0.2095238095238095,
 0.12857142857142856,
 0.0,
 0.32666666666666666,
 0.1,
 0.3,
 0.6,
 0.5,
 0.16666666666666666,
 0.02,
 0.0,
 0.23285714285714287,
 0.1,
 0.14285714285714285,
 0.1,
 0.2222222222222222,
 0.1,
 0.0,
 0.6366666666666667,
 0.1,
 0.4514285714285714,
 0.0,
 0.275,
 0.01111111111111111,
 0.22666666666666666,
 0.22666666666666666,
 0.0,
 0.16666666666666666,
 0.05,
 0.016666666666666666,
 0.26666666666666666,
 0.569047619047619,
 0.5,
 0.315,
 0.0,
 0.3,
 0.125,
 0.1,
 0.1,
 0.2095238095238095,
 0.15,
 0.2,
 0.0,
 0.2095238095238095,
 0.0,
 0.01,
 0.1,
 0.22999999999999998,
 0.15,
 0.16666666666666666,
 0.5380

In [ ]:
# модель, которая берет топ-10 продуктов у n схожих пользователей (выбрана как итоговая)
# k - число продуктов, которые берем из всех предсказанных (k самых вероятных)

# результаты для модели с decomp_dim=35 и было посчитано всего топ-30 схожих пользователей

#_________________________________________
# среднее по 10 пользователям

# если учитывались 500/500 схожих результат был 
# для k=5 - 0.188
# для k=10 - 0.124
# для k=30 - 0.067
# full - 0.082

# если учитывались 200/500 схожих результат был 
# для k=5 - 0.185
# для k=10 - 0.120
# для k=30 - 0.065
# full - 0.079

# если учитывались 30/30 схожих результат был 
# для k=5 - 0.176
# для k=10 - 0.114
# для k=30 - 0.066

# если учитывались 10/30 схожих результат был 
# для k=5 - 0.187
# для k=10 - 0.117
# для k=30 - 0.056

# если учитывались 5/30 схожих результат был 
# для k=5 - 0.15
# для k=10 - 0.09
# для k=30 - 0.05

#_________________________________________
# среднее по 100 пользователей, когда не учитывался сам юзер, но менялся размер топа продуктов у схожих

# топ-5 продуктов у схожих
# если учитывались 50/500 схожих результат был 
# для k=1 - 0.56
# для k=5 - 0.264
# для k=10 - 0.156
# для k=30 - 0.078
# full - 0.065

# топ-10 продуктов у схожих
# если учитывались 50/500 схожих результат был 
# для k=1 - 0.54
# для k=5 - 0.263
# для k=10 - 0.163
# для k=30 - 0.082
# full - 0.070

# топ-15 продуктов у схожих
# если учитывались 50/500 схожих результат был 
# для k=1 - 0.52
# для k=5 - 0.259
# для k=10 - 0.161
# для k=30 - 0.084
# full - 0.073


#_________________________________________
# среднее по 1000 пользователей, когда не учитывался сам юзер

# если учитывались 200/500 схожих результат был 
# для k=5 - 0.240
# для k=10 - 0.157
# для k=30 - 0.09
# full - 0.077

# если учитывались 150/500 схожих результат был 
# для k=5 - 0.241
# для k=10 - 0.156
# для k=30 - 0.089
# full - 0.075

# если учитывались 50/500 схожих результат был 
# для k=1 - 0.51
# для k=5 - 0.246
# для k=10 - 0.156
# для k=30 - 0.087
# full - 0.0717

# если учитывались 30/500 схожих результат был 
# для k=5 - 0.242
# для k=10 - 0.151
# для k=30 - 0.083
# full - 0.067

# если учитывались 5/500 схожих результат был 
# для k=5 - 0.210
# для k=10 - 0.12
# для k=30 - 0.068
# full - 0.054

#_________________________________________
# среднее по 1000 пользователей, когда учитывался сам юзер

# если учитывались 50/500 схожих результат был 
# для k=5 - 0.2549
# для k=10 - 0.167
# для k=30 - 0.100
# full - 0.089

In [47]:
mapk(predicted.values(), actual.values(), k=1)

# модель, которая учитывала все продукты по таблице с 0/1
# k - число продуктов, которые берем из всех предсказанных (k самых вероятных)

# результаты для модели с decomp_dim=15 и было посчитано всего топ-30 схожих пользователей

#_________________________________________
# среднее по 100 пользователям

# если учитывались 30/30 схожих результат был 
# для k=5 - 0.23
# для k=10 - 0.15
# для k=30 - 0.082

# если учитывались 10/30 схожих результат был 
# для k=5 - 0.24
# для k=10 - 0.16
# для k=30 - 0.098

# если учитывались 5/30 схожих результат был 
# для k=5 - 0.26
# для k=10 - 0.18
# для k=30 - 0.12

# то есть чем больше схожих юзеров учитываем, тем меньше точность предсказания


#_________________________________________
# рассмотрим среднее по 1000 пользователей

# если учитывались 3/30 схожих результат был 
# для k=1 - 0.524
# для k=5 - 0.317
# для k=10 - 0.246
# для k=30 - 0.17

# если учитывались 5/30 схожих результат был 
# для k=1 - 0.507
# для k=5 - 0.26
# для k=10 - 0.20
# для k=30 - 0.14

# если учитывались 15/30 схожих результат был 
# для k=1 - 0.498
# для k=5 - 0.229
# для k=10 - 0.15
# для k=30 - 0.097

# неплохое значение для первого предсказанного продукта (самый вероятный, который выдала модель)

# возможно увеличение размерности декомпозиции изменит тенденцию увеличения точности при уменьшении кол-ва учитываемых соседей

0.524

In [37]:
mapk(predicted.values(), actual.values(), k=10)

# модель, которая учитывала все продукты по таблице с 0/1
# k - число продуктов, которые берем из всех предсказанных (k самых вероятных)

# результаты для модели с decomp_dim=20 и было посчитано всего топ-30 схожих пользователей

#_________________________________________
# среднее по 100 пользователям

# если учитывались 30/30 схожих результат был 
# для k=1 - 0.56
# для k=5 - 0.239
# для k=10 - 0.151
# для k=30 - 0.08

# если учитывались 10/30 схожих результат был 
# для k=1 - 0.56
# для k=5 - 0.249
# для k=10 - 0.159
# для k=30 - 0.95

# если учитывались 5/30 схожих результат был 
# для k=1 - 0.61
# для k=5 - 0.287
# для k=10 - 0.19
# для k=30 - 0.13

# то есть чем больше схожих юзеров учитываем, тем ниже точность предсказания


#_________________________________________
# рассмотрим среднее по 1000 пользователей

# если учитывались 1/30 схожих результат был 
# для k=1 - 0.661
# для k=5 - 0.480
# для k=10 - 0.391
# для k=30 - 0.279
# full - 0.216

# если учитывались 3/30 схожих результат был 
# для k=1 - 0.56
# для k=5 - 0.329
# для k=10 - 0.257
# для k=30 - 0.185
# full - 0.153

# если учитывались 5/30 схожих результат был 
# для k=1 - 0.518
# для k=5 - 0.284
# для k=10 - 0.213
# для k=30 - 0.150
# full - 0.128

# если учитывались 10/30 схожих результат был 
# для k=1 - 0.508
# для k=5 - 0.245
# для k=10 - 0.172
# для k=30 - 0.113
# full - 0.103

# если учитывались 15/30 схожих результат был 
# для k=1 - 0.504
# для k=5 - 0.24
# для k=10 - 0.162
# для k=30 - 0.101
# full - 0.0937

# если учитывались 30/30 схожих результат был 
# для k=1 - 0.505
# для k=5 - 0.233
# для k=10 - 0.152
# для k=30 - 0.092
# full - 0.084


# неплохое значение для первого предсказанного продукта (самый вероятный, который выдала модель)

# возможно увеличение размерности декомпозиции изменит тенденцию увеличения точности при уменьшении кол-ва учитываемых соседей

0.1726690058893928

In [ ]:
mapk(predicted.values(), actual.values(), k=10)

# модель, которая учитывала все продукты по таблице с 0/1
# k - число продуктов, которые берем из всех предсказанных (k самых вероятных)

# результаты для модели с decomp_dim=35 и было посчитано всего топ-30 схожих пользователей


# чем больше схожих юзеров учитываем, тем ниже точность предсказания


#_________________________________________
# рассмотрим среднее по 1000 пользователей


# если учитывались 1/30 схожих результат был 
# для k=1 - 0.662
# для k=5 - 0.4812
# для k=10 - 0.3921
# для =15 - 0.3450
# для k=30 - 0.2803
# full - 0.2174

# если учитывались 3/30 схожих результат был 
# для k=1 - 0.556
# для k=5 - 0.3420
# для k=10 - 0.2694
# для k=15 - 0.2345
# для k=30 - 0.1911
# full - 0.1568

# если учитывались 5/30 схожих результат был 
# для k=1 - 0.54
# для k=5 - 0.2955
# для k=10 - 0.2230
# для k=15 - 0.1911
# для k=30 - 0.1558
# full - 0.1328

# если учитывались 10/30 схожих результат был 
# для k=1 - 0.528
# для k=5 - 0.2621
# для k=10 - 0.1862
# для k=15 - 0.1536
# для k=30 - 0.1197
# full - 0.1068

# если учитывались 15/30 схожих результат был 
# для k=1 - 0.524
# для k=5 - 0.2565
# для k=10 - 0.1742
# для k=15 - 0.1411
# для k=30 - 0.1079
# full - 0.0977

# если учитывались 30/30 схожих результат был 
# для k=1 - 0.515
# для k=5 - 0.251
# для k=10 - 0.163
# для k=30 - 0.092


# неплохое значение для первого предсказанного продукта (самый вероятный, который выдала модель)

# возможно увеличение размерности декомпозиции изменит тенденцию увеличения точности при уменьшении кол-ва учитываемых соседей

# Summary

Ускорены методы fit(), get_similarity(), predict(), get_data_to_check_metrics()

fit():
   + вынесена self.decompose() для выполнения только в том случае, если нужно полное обучение модели
   
get_similarity():
   + ранее считал для размерности декомпозиции 10 около 3 часов, сейчас это занимает порядка полутора часов (ускорилось благодаря счету не каждого юзера отдельно, а по 100 за раз, те матричные операции выполняются быстрее)
   + теперь считает как номера пользователей, так и их веса (те теперь считается и сохраняется 2 датафрейма, как рез-т обучения)
   + использован другой способ сортировки данных (полагаю это тоже ускорило процесс)
   
predict():
   + Из цикла вынесена операция from_coo, занимавшая порядка секунды, вытащить из уже посчитанной структуры необходимые данные по индексу уже сущие копейки
   + Не считает заново веса, так как они теперь считаются в get_similarity()
   + Исходя из параметров может посчитать для одного юзера с опеределенным id/для всех юзеров/для части юзеров, начиная с 1; также есть параметр, указывающий сколько из топа схожих юзеров надо учесть
   + По итогу на весь датасет, на который ранее требовалось 4500 часов, сейчас требуется порядка 7 часов, если уменьшить число пользователей, по которым сделаем итоговую оценку, то управится со всеми данными примерно за полтора часа для 5 и за 2.5 часа для 10.
   
get_data_to_check_metrics():
   + Ускорен засчет оптимизации вышеописанных методов
   
Можем рассчитать любой топ пользователей, но метрика оказывается больше, если учитываем меньшее число схожих юзеров в predict(). Численные рез-ты полученные при подсчете метрик и некоторые комментарии в п. Метрички (см. комментарии предыдущей ячейки)



#### Для user-based модели выбраны гиперпараметры:
   + Число схожих юзеров - 50
   + Размер декомпозиции - 35
   + Размер топа продуктов схожих пользователей - 10
   
Изменения в модели: ранее учитывала только факт покупки и брала все предметы, купленные схожими юзерами из топа, теперь рассматривает только k популярных продуктов (исходя из кол-ва купленного) у схожих юзеров из топа и НЕ учитывает самого юзера для которого делается предсказание (формулировка по факту - рекомендуем то, что покупает чаще всего похожие на вас пользователи)